## Getting a page width heuristic for the mesh->instance volume

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
from pathlib import Path
from synthetic_pages.types.homogeneous_transform import HomogeneousTransform
from synthetic_pages.types.mesh import Mesh
from synthetic_pages.types.nrrd import Nrrd
import matplotlib.pyplot as plt
import numpy as np

labels_dir = Path("/Volumes/lachlan-ssd/Scroll1_labelled")
list(labels_dir.iterdir())

position_header = "03840_02048_02560"

vol = Nrrd.from_file(labels_dir/f'{position_header}_volume.nrrd')

matrix = np.array([
    [0, 0, 1, 0],  # X -> Z
    [0, 1, 0, 0],  # Y -> Y 
    [1, 0, 0, 0],  # Z -> X
    [0, 0, 0, 1]
])
xyz_to_zyx = HomogeneousTransform(matrix)
meshes = [xyz_to_zyx.apply(Mesh.from_obj(labels_dir/fp)) for fp in labels_dir.glob(f'{position_header}_volume_mesh_*.obj')]

In [4]:
import matplotlib.pyplot as plt
import numpy as np

from synthetic_pages.utils import compute_signed_distances
papyrus_threshold = 25248


page_thickness = 30
def split_mesh_into_connected(*meshes: list[Mesh]):
    return [Mesh.from_trimesh(_m) for m in meshes for _m in m.as_trimesh(process=True).split(only_watertight=False)]

page_meshes = split_mesh_into_connected(*meshes)

In [5]:
# Convery Nrrd to point cloud
import numpy as np
def mask_as_pointcloud(
        mask, # (H, W, D)
):
    coordinates_zyx = np.argwhere(mask)
    coordinates_zyx_world = coordinates_zyx[:, None, :] @ vol.metadata['space directions'] + vol.metadata['space origin']
    return coordinates_zyx_world[:, 0, :]

In [6]:
pcl = mask_as_pointcloud(vol.volume > papyrus_threshold)

In [7]:
sdfs = np.array(
    [
        compute_signed_distances(
            pcl, 
            m, 
            distance_upper_bound = page_thickness/2) 
            for m 
            in page_meshes
    ]
).transpose((1,0))

In [8]:
page_labels = np.argmin(sdfs, axis=-1) + 1 # page labels are positive indices
page_labels[sdfs.min(axis=-1) > page_thickness / 2] = 0 # no page is the zero index

In [9]:
mask = np.zeros_like(vol.volume, dtype=np.uint8)

In [10]:
centred = pcl - vol.metadata['space origin']
voxel_space_coordinates = (pcl - vol.metadata['space origin'])[:, None, :] @ vol.metadata['space directions'].T
voxel_space_coordinates = voxel_space_coordinates.astype(np.uint32)

In [11]:
mask[voxel_space_coordinates[:, 0, 0], voxel_space_coordinates[:, 0, 1], voxel_space_coordinates[:, 0 , 2]] = page_labels

In [12]:
Nrrd(mask, metadata=vol.metadata).write(labels_dir/f"{position_header}_labels.nrrd")